In [1]:
model = "/home/fit/zhaijdzz/WORK/models/llama-3.1-8B-Instruct"
tp_size = 1

In [2]:
import os
from pathlib import Path

BASE_DIR = str(Path.cwd().parent)
model_name = model.split('/')[-1]
profiler_path = BASE_DIR + f"/profiler/profile_{model_name}_tp{tp_size}_online.npy"
profiler_path = "/home/fit/zhaijdzz/workspace/flowprefill/" + f"profiler/profile_{model_name}_tp{tp_size}_online.npy"
os.environ["VLLM_PROFILER_PATH"] = str(profiler_path)
print(f"VLLM_PROFILER_PATH: {profiler_path}")
from vllm import SamplingParams, AsyncEngineArgs, AsyncLLMEngine
from vllm.config import KVTransferConfig
os.environ['UCX_NET_DEVICES'] = 'all'

p_params = {
        "model": model,
        "max_model_len": 1024 * 24,

        "max_num_batched_tokens": 1024 * 32,
        "is_flowprefill": True,
        "num_runners": 128,

        "enforce_eager": True,
        "trust_remote_code": True,
        "gpu_memory_utilization": 0.8,

        "tensor_parallel_size": tp_size,
        "enable_chunked_prefill": True,
        "enable_prefix_caching": False,
        "dev_ranks": [0],

        "kv_transfer_config": KVTransferConfig(
            kv_connector="NixlConnector",
            kv_role="kv_both",
            kv_port="3600",
        ),
}

d_params = {
        "model": model,
        "max_model_len": 1024 * 24,

        "is_flowprefill": False,
        "enforce_eager": True,
        "trust_remote_code": True,
        "gpu_memory_utilization": 0.7,

        "tensor_parallel_size": tp_size,
        "enable_chunked_prefill": True,
        "enable_prefix_caching": False,
        "dev_ranks": [1],

        "kv_transfer_config": KVTransferConfig(
            kv_connector="NixlConnector",
            kv_role="kv_both",
            kv_port="3700",
        ),
}
p_engine_arg = AsyncEngineArgs(**p_params)
d_engine_arg = AsyncEngineArgs(**d_params)

VLLM_PROFILER_PATH: /home/fit/zhaijdzz/workspace/flowprefill/profiler/profile_llama-3.1-8B-Instruct_tp1_online.npy


/home/fit/zhaijdzz/miniconda3/envs/flowprefill/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BASE_DIR

'/home/fit/zhaijdzz/workspace/FlowPrefill'

In [4]:
p_asyllm = AsyncLLMEngine.from_engine_args(p_engine_arg)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 02-17 00:58:07 [model.py:638] Resolved architecture: LlamaForCausalLM
INFO 02-17 00:58:07 [model.py:1756] Using max model len 24576


2026-02-17 00:58:08,229	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 02-17 00:58:08 [scheduler.py:207] Chunked prefill is enabled with max_num_batched_tokens=32768.
WARNING 02-17 00:58:08 [vllm.py:589] Enforce eager set, overriding optimization level to -O0
INFO 02-17 00:58:08 [vllm.py:695] Cudagraph is disabled under eager mode
WARNING 02-17 00:58:08 [vllm.py:916] Turning off hybrid kv cache manager because `--kv-transfer-config` is set. This will reduce the performance of vLLM on LLMs with sliding window attention or Mamba attention. If you are a developer of kv connector, please consider supporting hybrid kv cache manager for your connector by making sure your connector is a subclass of `SupportsHMA` defined in kv_connector/v1/base.py.
(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:08 [core.py:106] Initializing a V1 LLM engine (v0.11.3.dev0+g275de3417.d20260214) with config: model='/home/fit/zhaijdzz/WORK/models/llama-3.1-8B-Instruct', speculative_config=None, tokenizer='/home/fit/zhaijdzz/WORK/models/llama-3.1-8B-Instruct', skip_tokenizer_init=F

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:02<00:06,  2.22s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:04<00:04,  2.20s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:06<00:02,  2.21s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:07<00:00,  1.57s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:07<00:00,  1.80s/it]
(EngineCore_DP0 pid=1241128) 


(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:18 [default_loader.py:308] Loading weights took 7.23 seconds
(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:18 [gpu_model_runner.py:3492] Model loading took 14.9889 GiB memory and 7.686155 seconds
(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:21 [gpu_worker.py:435] Available KV cache memory: 45.05 GiB
(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:21 [kv_cache_utils.py:1286] GPU KV cache size: 369,008 tokens
(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:21 [kv_cache_utils.py:1291] Maximum concurrency for 24,576 tokens per request: 15.01x
(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:21 [nixl_connector.py:71] NIXL is available
(EngineCore_DP0 pid=1241128) INFO 02-17 00:58:21 [factory.py:64] Creating v1 connector with name: NixlConnector and engine_id: ef7565ad-930f-4877-bc12-91443d1559ba
(EngineCore_DP0 pid=1241128) WARNING 02-17 00:58:21 [base.py:163] Initializing KVConnectorBase_V1. This API is experimental and subject to change in the futur

In [5]:
d_asyllm = AsyncLLMEngine.from_engine_args(d_engine_arg)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 02-17 00:58:39 [model.py:638] Resolved architecture: LlamaForCausalLM
INFO 02-17 00:58:39 [model.py:1756] Using max model len 24576
INFO 02-17 00:58:39 [scheduler.py:207] Chunked prefill is enabled with max_num_batched_tokens=2048.
WARNING 02-17 00:58:39 [vllm.py:589] Enforce eager set, overriding optimization level to -O0
INFO 02-17 00:58:39 [vllm.py:695] Cudagraph is disabled under eager mode
WARNING 02-17 00:58:39 [vllm.py:916] Turning off hybrid kv cache manager because `--kv-transfer-config` is set. This will reduce the performance of vLLM on LLMs with sliding window attention or Mamba attention. If you are a developer of kv connector, please consider supporting hybrid kv cache manager for your connector by making sure your connector is a subclass of `SupportsHMA` defined in kv_connector/v1/base.py.
(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:39 [core.py:106] Initializing a V1 LLM engine (v0.11.3.dev0+g275de3417.d20260214) with config: model='/home/fit/zhaijdzz/WORK/models/

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:02<00:07,  2.56s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:05<00:05,  2.78s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:07<00:02,  2.54s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  1.77s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  2.08s/it]
(EngineCore_DP0 pid=1241871) 


(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:51 [default_loader.py:308] Loading weights took 8.35 seconds
(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:51 [gpu_model_runner.py:3492] Model loading took 14.9889 GiB memory and 8.829197 seconds
(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:52 [gpu_worker.py:435] Available KV cache memory: 39.89 GiB
(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:52 [kv_cache_utils.py:1286] GPU KV cache size: 326,752 tokens
(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:52 [kv_cache_utils.py:1291] Maximum concurrency for 24,576 tokens per request: 13.30x
(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:52 [factory.py:64] Creating v1 connector with name: NixlConnector and engine_id: dcd11d56-e99f-47c5-aa2d-1190dadefd20
(EngineCore_DP0 pid=1241871) WARNING 02-17 00:58:52 [base.py:163] Initializing KVConnectorBase_V1. This API is experimental and subject to change in the future as we iterate the design.
(EngineCore_DP0 pid=1241871) INFO 02-17 00:58:52 [nixl_connect

In [6]:
from vllm.utils import random_uuid
import time
async def gen(asyllm, prompt, params, req_id):
    results_generator = asyllm.generate(prompt, params, req_id)
    final_output = None
    last_time = time.perf_counter()
    iter_times = []
    async for request_output in results_generator:
        now = time.perf_counter()
        iter_time = now - last_time
        iter_times.append(round(iter_time, 4))
        last_time = now
        final_output = request_output

    return iter_times, final_output

async def launch_prefill(asyllm, req_id, req_dict, idx, is_print=True):
    prompt = req_dict["prompt"]
    ttft_slo = req_dict["ttft_slo"]
    arrival = time.perf_counter()
    params = SamplingParams(temperature=0.0, max_tokens=1, 
                            extra_args={
                                "ttft_slo": ttft_slo,
                                "arrival": arrival,
                                'kv_transfer_params': {"do_remote_decode": True,}, # nixl
                                }
                            )
    ttfts, final_output = await gen(asyllm, prompt, params, req_id)
    prompt_len = len(final_output.prompt_token_ids)
    ttft = sum(ttfts) / len(ttfts)
    if is_print:
        print(f"req {idx}, prompt_len: {prompt_len}, arrival: {arrival:.4f}, ttft: {ttft:.4f}, ttft_slo: {ttft_slo}, attain: {ttft_slo >= ttft}")
    return final_output, {
            "num_tokens": prompt_len,
            "arrival": arrival,
            "ttft": ttft,
            "ttft_slo": ttft_slo,
            "ttft_attain": ttft_slo >= ttft,
        } 

async def launch_decode(asyllm, req_id, req_dict, idx, p_response, is_print=True): # nixl
    prompt = req_dict["prompt"]
    tbt_slo = req_dict["tbt_slo"]
    max_tokens = req_dict["output_length"]
    arrival = time.perf_counter()
    params = SamplingParams(temperature=0.0, max_tokens=max_tokens, 
                            extra_args={
                                "tbt_slo": tbt_slo,
                                }
                            )
    params.extra_args['kv_transfer_params'] = p_response.kv_transfer_params # nixl
    iter_times, final_output = await gen(asyllm, prompt, params, req_id)
    tbts = iter_times[1:]
    num_attain = sum([tbt < tbt_slo for tbt in tbts])
    tbt_attain = num_attain / len(tbts)
    avg_tbt = sum(tbts) / len(tbts)
    if is_print:
        print(f"req {idx}, output_length: {max_tokens}, avg_tbt: {avg_tbt:.4f}s, tbt_slo: {tbt_slo}, tbt attain: {tbt_attain:.4f}")
    return final_output, {
            "output_length": max_tokens,
            "tbt": avg_tbt,
            "tbt_slo": tbt_slo,
            "tbt_attain": tbt_attain,
            "iter_times": iter_times,
        }

async def launch_req(p_asyllm, d_asyllm, req_dict, idx, is_print=True): # nixl
    req_id = random_uuid()
    p_response, p_res_dict = await launch_prefill(p_asyllm, req_id, req_dict, idx, is_print=False)
    final_output, d_res_dict = await launch_decode(d_asyllm, req_id, req_dict, idx, p_response, is_print=False)
    
    result_dict = {
        **{
            "req_id": req_id,
            "type": req_dict["type"],
            "response": final_output.outputs[0].text
        },
        **p_res_dict,
        **d_res_dict
    }
    if is_print:
        print(f"req {idx}, prompt_len: {result_dict['num_tokens']}, output_length: {result_dict['output_length']}, "
        f"ttft: {result_dict['ttft']:.4f}, ttft_slo: {result_dict['ttft_slo']}, ttft attain: {result_dict['ttft_attain']}, "
        f"avg_tbt: {result_dict['tbt']:.4f}s, tbt_slo: {result_dict['tbt_slo']},, tbt attain: {result_dict['tbt_attain']:.2f}")
    return result_dict

In [7]:
async def warm_up():
    req_dict = {
        "prompt": "where is the capital of France? ", "output_length": 4, 
        "num_tokens": 7, "ttft_slo": 0.1, "tbt_slo": 0.1, "type": None}
    for idx in range(32):
        out = await launch_req(p_asyllm, d_asyllm, req_dict, idx, is_print=False)
await warm_up()

WARNING 02-17 00:59:00 [async_llm.py:296] Processor has been moved under OpenAIServing and will be removed from AsyncLLM in v0.13.


In [8]:
def res_print(results, req_rate, rate_scale, span, start_time):
    num_attain = sum([item['ttft_attain'] for item in results])
    slo_attainments = num_attain / len(results)
    cost_time = time.perf_counter() - start_time
    tbt_attains = [item['tbt_attain'] for item in results]
    tbt_attain = sum(tbt_attains) / len(tbt_attains)
    print(f"Req rate: {req_rate:.4f}, rate_scale: {rate_scale}, Trace Span: {span:.4f}s")
    print(f"cost_time: {cost_time:.4f}s")
    print(f"goodput: {num_attain / cost_time:.4f} req/s")
    print(f"TTFT slo attainment: {slo_attainments:.4f}, TBT slo attainment: {tbt_attain:.4f}")
    print(f"Avg TTFT: {sum([item['ttft'] for item in results]) / len(results):.4f}, Avg TBT: {sum([item['tbt'] for item in results]) / len(results):.4f}")
    # SLO Scaling results
    slo_scale_list = [round(i/10, 2) for i in range(4, 21, 2)]
    print(f"slo_scale_list: {slo_scale_list}")
    slo_result = []

    for slo_scale in slo_scale_list:
        num_attain = sum([item['ttft_slo']*slo_scale > item["ttft"] for item in results])
        slo_attainments = round(num_attain / len(results), 4)
        slo_result.append(slo_attainments)
    print(f"slo_attainment: {slo_result}")

In [9]:
SLO_DICT = {
    "text": (0.25, 0.1),
    "image": (0.5, 0.1),
    "search": (4.0, 0.2),
    "file": (6.0, 0.2),
}

In [16]:
import os, sys, asyncio
sys.path.insert(0, BASE_DIR)
from trace_build.trace import sample_reqs, bulid_workload
async def benchmark(rate_scale, trace_path, p_asyllm, d_asyllm):
    global SLO_DICT
    sampled_reqs, trace_data = sample_reqs(trace_path, SLO_DICT)
    # Qwen Trace
    num_sampled = len(sampled_reqs)
    arrivals, intervals = bulid_workload(trace_data, num_sampled, rate_scale=1 / rate_scale)
    
    intervals = [0]+intervals.tolist()
    span = arrivals[-1]-arrivals[0]
    req_rate = num_sampled / span
    print(f"--------- Req rate: {req_rate:.2f}, rate_scale: {rate_scale}, num reqs: {len(sampled_reqs)}, span: {span:.2f}s ---------")
    
    start_time = time.perf_counter()
    tasks = []
    for idx, req_dict in enumerate(sampled_reqs):
        await asyncio.sleep(intervals[idx])
        tasks.append(asyncio.create_task(launch_req(p_asyllm, d_asyllm, req_dict, idx, is_print=True)))
        # tasks.append(asyncio.create_task(launch_req(p_asyllm, d_asyllm, req_dict, idx, is_print=False)))
    span = time.perf_counter()-start_time
    
    results = await asyncio.gather(*tasks)
    res_print(results, req_rate, rate_scale, span, start_time)
    return results

In [36]:
rate_scale = 2.0
trace_path = BASE_DIR + "/trace_build/qwen_traceA_0.0min_2.0min.jsonl"
task = asyncio.create_task(benchmark(rate_scale, trace_path, p_asyllm, d_asyllm))
# results = await benchmark(rate_scale, trace_path, p_asyllm, d_asyllm)

avg input len: 1634.51, max length: 16744, num req: 327, span: 1.98 min
------- req_type: text num: 226 -------
input length: mean: 645.93, P99: 3256.00, Std: 629.71
------- req_type: image num: 24 -------
input length: mean: 393.29, P99: 807.56, Std: 112.88
------- req_type: search num: 66 -------
input length: mean: 5054.77, P99: 15127.45, Std: 3070.49
------- req_type: file num: 11 -------
input length: mean: 4131.91, P99: 10559.60, Std: 3518.73
--------- Req rate: 5.49, rate_scale: 2.0, num reqs: 327, span: 59.54s ---------
req 3, prompt_len: 360, output_length: 54, ttft: 0.0499, ttft_slo: 0.5, ttft attain: True, avg_tbt: 0.0157s, tbt_slo: 0.1,, tbt attain: 1.00
req 1, prompt_len: 380, output_length: 76, ttft: 0.0670, ttft_slo: 0.5, ttft attain: True, avg_tbt: 0.0156s, tbt_slo: 0.1,, tbt attain: 1.00
req 4, prompt_len: 758, output_length: 211, ttft: 0.0785, ttft_slo: 0.25, ttft attain: True, avg_tbt: 0.0253s, tbt_slo: 0.1,, tbt attain: 0.98
req 19, prompt_len: 861, output_length: 9

In [66]:
# debug
code = """
print("executing_req:", self.executing_req)
print("H_priority_req:", self.H_priority_req)
runner_id = self.executing_req.runner_id
print(self.preempted_signals[runner_id])
print(self.execution_signals[runner_id])
print(self.exe_lock, self.sch_upd_lock, self.sort_lock)
print("kv usage:", self.scheduler.kv_cache_manager.block_pool.get_usage())
"""
print(await p_asyllm.engine_core.call_utility_async("exec_code", code))

None


In [ ]:
results = await task